# Advanced multi index

Documentation sur le multi indexing

- https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

## Variables name

|  Variable name        |  Type   |
| --------------------- | ------- |
| ID                    |  object |
| Country_name          | object  |
| country_ref           | object  |
| Region                | object  |
| industry              | object  |
| A_reference           | float64 |
| A_market              | float64 |
| B_reference           | float64 |
| B_market              | float64 |
| C_reference           | float64 |
| C_market              | float64 |
| D_reference           | float64 |
| D_market              | float64 |
| E_reference           | float64 |
| E_market              | float64 |
| Total_reference       | float64 |
| Total_market          | float64 |

Note that, the market includes the reference

In [ ]:
import pandas as pd
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

In [ ]:
path = 'https://sgithub.fr.world.socgen/X196663/Introduction_Python/blob/' \
'master/Data/dataPandasClass.gz?raw=true'

df_final = pd.read_csv(path, compression='gzip')
df_final.tail()

In [ ]:
reference = [
    "A_reference",
    "B_reference",
    "C_reference",
    "D_reference",
    "E_reference",
]
market = [
    "A_market",
    "B_market",
    "C_market",
    "D_market",
    "E_market",
]

## Slicing the data

On peut faciliment filter la data avec l'index.

Dans Pandas, les index sont des tupples

In [ ]:
df_index = df_final.set_index(['ID','Country_name',
                           'country_ref', 'Region',
                   'industry'])
df_index.head()

Filtrage avec le premier index

In [ ]:
df_index.loc[('Customer 620')].head()

Filtrage avec le deuxième  index

In [ ]:
df_index.loc[(slice(None), 'France'), :].head()

Cross section indexing

In [ ]:
df_index.xs('L', level='industry').head()

### Creation multi indexation

In [ ]:
test = (df_final[['ID','Country_name',
                           'country_ref', 'Region', 'industry'] + reference + market]
 .set_index(['ID','Country_name',
                           'country_ref', 'Region',
                   'industry'])
 .transpose()
 .reset_index()
 .assign(origin = lambda x: x["index"].str.extract(r"(_[^_]+$)"),
        product = lambda x: x["index"].str.extract(r"(^[^_]+(?=_))"))
 .set_index(['origin', 'product'], append = True)
 .drop(columns = 'index')
 .reset_index(0, drop = True)
 .T
)
test.head()

### Slicing the data: index en ligne et colonne

On peut faciliment filter la data avec l'index

In [ ]:
test.loc[([slice(None), 'Australia'])].head()

On peut recuperer uniquement les colonnes references

In [ ]:
test.loc[:,("_reference")]

On peut recuperer les colonnes references pour la France uniquement

In [ ]:
test.loc[(slice(None), 'France'),("_reference")]

## Industry Market size by product

**the market share :Revenue to potential**

$$\text{Market share} = \sum  \text{Revenue Reference} / \sum \text{Total Market} $$

Nous avons, pour chaque industrie, 5 produits differents. Dans l'exercice ci dessous, l'objectif est de calculer les parts de marché de chaque industrie pour l'ensemble des 5 produits. Pour éviter de créer de nombreux objets, nous allons faire le calcul en une seule ligne de code.

Avant de d'englober le code, il est préférable de vous montrer le step by step

In [ ]:
df_final.head()

Step 1: Recuperer les variables que l'on a besoin et aggreger sur les industries.

Vous noterez que l'index est maintenant `industry`. Pour faciliter les étapes suivantes, il est préférable de transposer la matrice. Ainsi, chaque ligne a deux références, l'industrie et `market` ou industrie et `reference`

In [ ]:
step1 = (
    df_final[['industry'] + reference + market]
    .groupby('industry')
    .sum(axis = 0)
    .T
    .sort_index()
    
)
step1.head()

Comme nous avons besoin de faire la distinction entre reference et market (ie mettre en index soit reference soit market), on doit tout d'abord créer une nouvelle variable indiquant soit `market` ou `reference`

In [ ]:
step2 = (step1
         .reset_index()
         .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
        product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
         
)
step2.head()

Maintenant que nous avons créée nos deux nouvelles variables, on peut les mette dans l'index et faire l'aggregate directement sur `product`

In [ ]:
step3 = (step2
        .drop(columns = 'index')
        .sort_values(by = "product")
        .set_index(['product', 'origin'])
        )
step3.head()

Nous voila prèt pour faire l'aggrégation. En un sens, nous pouvons simplement grouper le calcul sur `product` et faire `_reference` / `_market` .  `_market` étant le shift dans le groupe

Par exemple pour le produit A, industrie A, le calcul est: 4.907946/52.405622n ou  x/x shift 1 (forward).

Notez que pour les cellules `_market`, l'aggregation va retourner que des nan's. On les supprime.

In [ ]:
step4 = (step3
         .groupby(level=0)
         .apply(lambda x:  x / x.shift(1))
         .dropna()
         .droplevel('origin')
        )

step4

le calcul des parts de marché étant fini, on va pouvoir améliorer la table en calculant le total du marché par industrie, et mettre un conditional formating

In [ ]:
(step4
.transpose()
 .assign(total_industry = lambda x: x.sum(1))
 .sort_values(by ='total_industry')
 #.drop(columns = 'total_industry')
 .style
 .format("{:.2%}")
.background_gradient(subset = ['A','B', 'C',
                              'D', 'E'],cmap=cm)

)

On peut tout regrouper en une seule ligne de code

In [ ]:
(df_final[['industry'] + reference + market]
 .groupby('industry')
 .sum(axis = 0)
 .transpose()
 .reset_index()
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
        product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
 .drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
 .groupby(level = 0)
 .apply(lambda x: x.shift(1) / x )
 .dropna()
 .droplevel('origin')
 .transpose()
 .assign(total_industry = lambda x: x.sum(1))
 .sort_values(by ='total_industry')
 .drop(columns = 'total_industry')
 .style
 .format("{:.2%}")
 .background_gradient(subset = ['A','B', 'C',
                              'D', 'E'],cmap=cm)
)

# Base de modélisation

Variables name

- 'top_def_12_mois': top defaut
- 'nb_op_rej_dar_nn_paye_cons': nombre d operations rejetées (dépassement autorisation découvert)
- 'MNT_MVT_DEB_MOIS_max': Montant mouvement compte débiteur, parmis tous ses comptes a vue
- 'sld_moy_net_sum': somme soldes moyens sur tous les cav
- 'ratio_imp_enc': ratio impayés/encours
- 'somme_epar_MVT_CR_MOIS': somme des mouvements créditeurs sur comptes éparges
- 'somme_epar_encours': sommes comptes épargnes encours
- 'duree_app': nombre de jours depuis la prévension (en suveillance)
- 'anc_rel: ancienneté de la relation

In [ ]:
path  = 'https://sgithub.fr.world.socgen/X196663/Introduction_Python/blob/' \
'master/Data/base_modelisation.gz?raw=true'

df = (pd.read_csv('base_modelisation.csv',
                  compression='gzip',
                  parse_dates=['DAT_ANNEE_MOIS'],
                  #index_col=['numpers',
                  #           'DAT_ANNEE_MOIS']
                 )
      .drop(columns='Unnamed: 0')
      .assign(month = lambda x: x["DAT_ANNEE_MOIS"].dt.month,
              year = lambda x: x["DAT_ANNEE_MOIS"].dt.year,
              month_year = lambda x: x["DAT_ANNEE_MOIS"].dt.strftime('%m-%Y'),
             )
      .sort_values(by = ['numpers', 'DAT_ANNEE_MOIS'])
      .set_index(['numpers',
                  #'DAT_ANNEE_MOIS',
                  #'month_year','year','month'
                 ])
      
     )
df.head()

## Nombre de jours entre deux selections

In [ ]:
df = pd.concat(
    [
        (df
         .groupby(level = 0)['DAT_ANNEE_MOIS']
         .diff()
         .rename('days_diff_quarter')
         .dt
         .days/120),
        df
    ],
    axis = 1
)

In [ ]:
df.head()

In [ ]:
df.describe()

## Multi indexation Pandas

La multi indexation dans Pandas permet de faire de la minupulation de la data et des calcules assez simplement.

Tout d'abords, nous pouvons constater que les variables `numpers` et `DAT_ANNEE_MOIS` sont en index. Autrement dit, ses deux variables représentent les identifiants

In [ ]:
## Nombre unique individu
df.reset_index().groupby('numpers')['numpers'].nunique().sum()

In [ ]:
(df
 .groupby(level = [0])['top_def_12_mois']
 .count()
 .reset_index(name = 'count')
 .groupby('count')
 .count()
 #.plot
 #.barh()
)

## Nombre de quarters entre deux selections

1. Recuperer la liste des multis ID
2 Selectionner les multis ID
3. Calcule les averages quarters par nombre d'occurence

In [ ]:
import numpy as np

In [ ]:
multi = (df
 .groupby(level = [0])['top_def_12_mois']
 .count()
 .loc[lambda x : x > 1]
 .index
)

multi

In [ ]:
df_multi = df.loc[lambda x: (x.index.isin(multi)) & 
                 (x["days_diff_quarter"].notna())]

df_multi.head()

In [ ]:
(df_multi
 .groupby(['month_year'])['days_diff_quarter']
 .agg(['count' ,np.mean, np.max]
 )
)

## Basic filling operation

Pour chaque individu, on crée les dates manquantes. Les nouvelles dates vont etre remplies de nan. Une possibilité peut etre de calculer la moyenne des MNT_MVT_DEB_MOIS_max par mois, et on remplit les nan par cette moyenne..

On va faire une approche step by step puis dans un second temps, une approche par chain

### Step by Step

1)  multi index

In [ ]:
step1 = df_multi.set_index('month_year',append=True)['MNT_MVT_DEB_MOIS_max']
step1.head()

2)  Création des nouvelles dates pour tout le monde

In [ ]:
step2 = step1.unstack(-1)
step2.head()

3) Reshape les colonnes en ligne en vue du remplissage

In [ ]:
step3 = step2.stack(dropna = False)
step3.head()

4) Remplissage avec la moyenne mensuelle

In [ ]:
step4 = step3.groupby(level = 1).apply(lambda x: x.fillna(x.mean()))
step4.head()

### Chaining approach

In [ ]:
(df_multi
 .set_index('month_year',append=True)['MNT_MVT_DEB_MOIS_max']
 .unstack(-1)
 .stack(dropna = False)
 .groupby(level = 1)
 .apply(lambda x: x.fillna(x.mean()))
).head()

## Basic summary stat

In [ ]:
df.groupby(['month_year']).agg([np.mean, np.std]).T

In [ ]:
(df
 .groupby(['month_year'])['sld_moy_net_sum']
 .agg([np.mean, np.std, np.min, np.max])
 .style
 .bar( color='#d65f5f')
)

## Export Excel avec Styling

In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [ ]:
(df
 .groupby(['month_year'])['sld_moy_net_sum']
 .agg([np.mean, np.std, np.min, np.max])
 .style.\
    applymap(color_negative_red).\
    apply(highlight_max).\
    to_excel('styled.xlsx', engine='openpyxl')
)

## Library `xlwings`

In [ ]:
import xlwings as xw
import seaborn as sns

In [ ]:
wb = xw.Book('styled.xlsx')
sht = wb.sheets['Sheet1']

In [ ]:
sht.range('A10').value = df[:10]

In [ ]:
df_test = (df
 .loc[lambda x : (x['sld_moy_net_sum'] > 0) &
      (x['somme_epar_MVT_CR_MOIS'] > 0)]
 .assign(ln_revenue = lambda x :x["sld_moy_net_sum"] *100,
          ln_saving = lambda x :x["somme_epar_MVT_CR_MOIS"],
         )
)
df_test

Export graph to Excel

In [ ]:
import matplotlib.pyplot as plt
ax = sns.scatterplot(x="ln_revenue",
                     y="ln_saving",
                     data=df_test)

fig = ax.get_figure()

sht.pictures.add(fig, name='MyPlot', update=True)

Import Excel to Pandas

In [ ]:
pd.DataFrame(xw.Range(xw.Range('A11'), xw.Range('O20')).value,
            columns = xw.Range(xw.Range('A10'), xw.Range('O10')).value)